# RAG 성능 평가

---


## 환경 설정 및 준비


`(1) Env 환경변수`


In [ ]:
from dotenv import load_dotenv

load_dotenv()

`(2) 기본 라이브러리`


In [ ]:
import os
from glob import glob

from pprint import pprint
import json

`(3) Langsmith tracing 설정`


In [ ]:
# Langsmith tracing 여부를 확인 (true: langsmith 추척 활성화, false: langsmith 추척 비활성화)
import os

print(os.getenv("LANGSMITH_TRACING"))

---

## RAG 시스템 성능 평가

- **RAG 기술의 핵심**: 외부 지식 검색과 LLM 결합으로 응답 품질 향상

- **평가 기준**: LLM-as-judge 방식으로 사실성, 관련성, 충실도, 유용성 평가

- **체계적인 A/B 테스트**: 각 컴포넌트별 성능 비교 및 영향도 분석으로 최적 구성 도출

- **평가 방법론**: 오프라인(참조답변 기반), 온라인(실시간), 페어와이즈(비교) 평가 구분

<center>
<img src="https://raw.githubusercontent.com/tsdata/image_files/main/202505/rag_evaluation.png" alt="rag" align="center" border="0"  width="1000" height=auto>
</center>


---

### **평가 대상(Evaluation Target)**

- **검색(Retrieval)** 단계:

  1. 관련 문서와 쿼리 간의 연관성(Relevance)을 통해 검색된 문서가 쿼리의 정보 요구를 얼마나 잘 충족하는지 평가
  1. 관련 문서와 후보 문서 간의 정확성(Accuracy)을 통해 시스템이 적절한 문서를 식별하는 능력을 측정

- **생성(Generation)** 단계:

  1. 응답과 쿼리의 연관성(Relavance)
  1. 응답과 관련 문서 간의 충실도(Faithfulness)
  1. 응답과 샘플 응답 간의 정확성(Correctness)

- 추가 고려사항:
  - **핵심 성능 지표**: Latency(응답 속도), Diversity(검색 다양성), Noise Robustness(잡음 내구성)
  - **안전성 평가**: Negative Rejection(불충분 정보 거부), Counterfactual Robustness(오정보 식별)
  - **사용자 경험**: Readability(가독성), Toxicity(유해성), Perplexity(복잡성) 등 추가 고려

<center>
<img src="https://raw.githubusercontent.com/tsdata/image_files/main/202505/rag_evaluation_target.png"  alt="rag" align="center" border="0"  width="800" height=auto>
</center>

[출처] https://arxiv.org/abs/2405.07437


---

### **평가 데이터셋 구축(Evaluation Dataset)**

- **데이터셋 구성 방식**: LLM 기반 새로운 데이터셋 생성 (Synthetic Data)

- **맞춤형 데이터셋** 구축으로 RAG 시스템의 실용성 평가 강화

- [실습] : **Ragas** (https://docs.ragas.io/en/stable/) 활용

  - **RAG 시스템 평가**를 위한 오픈소스 프레임워크
  - **주요 지표**: Faithfulness, Answer Relevancy, Context Relevancy 등 평가
  - **실용성**: 자동화된 평가 파이프라인 구축 가능


`(1) LangChain 문서 준비`


In [ ]:
from langchain_community.document_loaders import TextLoader


# 데이터 로드
def load_text_files(txt_files):
    data = []

    for text_file in txt_files:
        loader = TextLoader(text_file, encoding="utf-8")
        data += loader.load()

    return data


korean_txt_files = glob(os.path.join("../data", "*_KR.md"))
korean_data = load_text_files(korean_txt_files)

print("Korean data:")
pprint(korean_data)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 문장을 구분하여 분할 - 정규표현식 사용 (문장 구분자: 마침표, 느낌표, 물음표 다음에 공백이 오는 경우)
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base",  # TikToken 인코더 이름
    separators=["\n\n", "\n", r"(?<=[.!?])\s+"],  # 구분자
    chunk_size=300,
    chunk_overlap=0,
    is_separator_regex=True,  # 구분자가 정규식인지 여부
    keep_separator=True,  # 구분자 유지 여부
)

korean_docs = text_splitter.split_documents(korean_data)

print("한국어 문서 수:", len(korean_docs))
print("-" * 100)
print(korean_docs[0].metadata)
pprint(korean_docs[0].page_content)
print("-" * 100)
print(korean_docs[1].metadata)
pprint(korean_docs[1].page_content)

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# OpenAI Embeddings 모델을 로드
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# Chroma 벡터 저장소 생성하기
vector_store = Chroma.from_documents(
    documents=korean_docs,
    embedding=embedding_model,
    collection_name="db_korean_cosine",
    persist_directory="../chroma_db",
    collection_metadata={"hnsw:space": "cosine"},  # l2, ip, cosine 중에서 선택
)

# 결과 확인
print(f"저장된 Document 개수: {len(vector_store.get()['ids'])}")

`(2) LLM 설정`


In [ ]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

# LLM과 임베딩 모델 초기화
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1", temperature=0.2))
generator_embeddings = LangchainEmbeddingsWrapper(
    OpenAIEmbeddings(model="text-embedding-3-small")
)

`(3) Test Data 생성`

- uv add rapidfuzz 설치 필요


In [ ]:
from ragas.testset.persona import Persona

# 페르소나 정의 (다양한 관점에서 질문 생성)
personas = [
    Persona(
        name="graduate_researcher",  # 박사과정 연구원: 심도 있는 분석적 질문
        role_description="미국 전기차 시장을 연구하는 한국인 박사과정 연구원으로, 전기차 정책과 시장 동향에 대해 깊이 있는 분석을 하고 있습니다. 한국어만을 사용합니다.",
    ),
    Persona(
        name="masters_student",  # 석사과정 학생: 개념 이해를 위한 질문
        role_description="전기차 산업을 공부하는 한국인 석사과정 학생으로, 미국 전기차 시장의 기초적인 개념과 트렌드를 이해하려 노력하고 있습니다. 한국어만을 사용합니다.",
    ),
    Persona(
        name="industry_analyst",  # 산업 분석가: 실무 중심적 질문
        role_description="한국 자동차 회사에서 미국 전기차 시장을 분석하는 주니어 연구원으로, 실무적인 시장 데이터와 경쟁사 동향에 관심이 많습니다. 한국어만을 사용합니다.",
    ),
    Persona(
        name="undergraduate_student",  # 학부생: 기초적인 학습 질문
        role_description="자동차 공학을 전공하는 한국인 학부생으로, 미국 전기차 기술과 시장에 대해 기본적인 지식을 습득하고자 합니다. 한국어만을 사용합니다.",
    ),
]

In [ ]:
from ragas.testset import TestsetGenerator

# TestsetGenerator 생성
generator = TestsetGenerator(
    llm=generator_llm, embedding_model=generator_embeddings, persona_list=personas
)

# 합성 데이터 생성
dataset = generator.generate_with_langchain_docs(korean_docs, testset_size=5)

In [ ]:
dataset.to_pandas()

In [ ]:
# 데이터 저장
dataset.to_pandas().to_csv("./data/ragas_testset.csv", index=False)

---

### **평가 지표(Evaluation Metric)**

#### 1) **검색(Retrieval) 평가**

- **Non-Rank Based Metrics**: Accuracy, Precision, Recall@k 등을 통해 관련성의 이진적 평가를 수행

- **Rank-Based Metrics**: MRR(Mean Reciprocal Rank), MAP(Mean Average Precision)를 통해 검색 결과의 순위를 고려한 평가를 수행

- **RAG 특화 지표**: 기존 검색 평가 방식의 한계를 보완하는 LLM-as-judge 방식 도입

- **포괄적 평가**: 정확도, 관련성, 다양성, 강건성을 통합적으로 측정

#### 2) **생성(Generation) 평가**

- **전통적 평가**: ROUGE(요약), BLEU(번역), BertScore(의미 유사도) 지표 활용

- **LLM 기반 평가**: 응집성, 관련성, 유창성을 종합적으로 판단하는 새로운 접근법 도입 (전통적인 참조 비교가 어려운 상황에서 유용)

- **다차원 평가**: 품질, 일관성, 사실성, 가독성, 사용자 만족도를 포괄적 측정

- **상세 프롬프트**와 **사용자 선호도** 기준으로 생성 텍스트 품질 평가


`(1) RAG 체인 - 평가 대상`


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 벡터 저장소 검색기 생성
retriever = vector_store.as_retriever(search_kwargs={"k": 2})

# RAG 체인
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# 템플릿 생성
template = """Answer the question based only on the following context:
{context}

Question: {query}
"""
prompt = ChatPromptTemplate.from_template(template)

qa_chain = prompt | llm | StrOutputParser()


def format_docs(relevant_docs):
    return "\n".join(doc.page_content for doc in relevant_docs)


query = "Tesla는 언제 누가 만들었나?"

relevant_docs = retriever.invoke(query)
qa_chain.invoke({"context": format_docs(relevant_docs), "query": query})

`(2) 평가 수행을 위한 데이터셋 전처리`


In [ ]:
# 데이터 로드
import pandas as pd

testset = pd.read_excel("../data/testset.xlsx")

# 데이터 확인
testset.head()

In [ ]:
from ragas import EvaluationDataset

# 데이터셋 생성
dataset = []

# 각 행에 대해 RAG 체인을 호출하여 결과를 저장
for row in testset.itertuples():
    query = row.user_input  # 사용자 입력
    reference = row.reference  # 참조 답변
    relevant_docs = retriever.invoke(query)  # 검색된 문서
    response = qa_chain.invoke(  # RAG 체인 생성 답변 생성
        {
            "context": format_docs(relevant_docs),
            "query": query,
        }
    )

    dataset.append(
        {
            "user_input": query,
            "retrieved_contexts": [d.page_content for d in relevant_docs],
            "response": response,
            "reference": reference,
        }
    )

evaluation_dataset = EvaluationDataset.from_list(dataset)

In [ ]:
# 데이터프레임 변환하여 확인
evaluation_dataset.to_pandas().head()

In [ ]:
# 데이터 저장
evaluation_dataset.to_pandas().to_csv("data/evaluation_dataset.csv", index=False)

`(3) 평가 수행`

- **LLMContextRecall**

  - 검색된 컨텍스트가 정답을 생성하는 데 필요한 **모든 정보를 포함**하고 있는지 평가

- **Faithfulness**

  - 생성된 응답이 검색된 컨텍스트와 얼마나 **사실적으로 일관**되는지를 측정

- **FactualCorrectness**
  - 생성된 응답과 reference의 **사실적 정확**성을 비교하고 평가


In [ ]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness
from langchain_openai import ChatOpenAI

# LLM 래퍼 생성
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
evaluator_llm = LangchainLLMWrapper(llm)

# 평가
result = evaluate(
    dataset=evaluation_dataset,  # 평가 데이터셋
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()],  # 평가 메트릭
    llm=evaluator_llm,  # LLM 래퍼
)

result

In [ ]:
# 결과를 데이터프레임으로 변환
result.to_pandas()

In [ ]:
# 데이터프레임 저장
result.to_pandas().to_csv("data/evaluation_result.csv", index=False)